In [14]:
import filter_reviews, semantic_segmentation
from semantic_segmentation import merge_short_sentences
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
import re


############ This has a limitation for the cases when the point enumeration becomes more than 9
############ This also have an issue with examples like "3.2.1" as it will be considered as a point
def split_into_points(text):
    splits_positions = []
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)  
    for i, char in enumerate(text):
        # match points that starts with:  - , * , = followed by a space
        if char in ['-','*', '='] and text[i+1] == ' ':
            if i > 0:
                # make sure this point is preceded by a sentence ending or new line
                if  text[i-1] in ['.','!','?','\n'] or text[i-2] in ['.','!','?','\n']:
                    splits_positions.append(i)
            else:
                splits_positions.append(i)
        if char.isnumeric():

            ## check for x.x and x.x.x: 
            if text[i+1] == '.' and text[i+2].isnumeric():
                i = i+2
                continue
            
                
            if text[i+1] in [')','.',':'] and text[i+2] != '\n':
                if i > 1:
                    # make sure this point is preceded by a sentence ending or new line
                    if  text[i-1] in [' ','\n'] or text[i-2] in ['.','!','?','\n']:
                        splits_positions.append(i)
                else:
                    splits_positions.append(i)
            ## Match when we have a number at the begining of a new line 
            elif (i == 0 or text[i-1] == '\n') and text[i+1] == ' ':
                splits_positions.append(i)

    splits_positions = [0] + splits_positions 
    points = [ text[splits_positions[i]:splits_positions[i+1]].strip() for i in range(len(splits_positions)-1)]
    points.append(text[splits_positions[-1]:].strip())

    points = [point for point in points if point]
    return points
# # Example usage:
# # paragraph = " Attention attribution and entropy terms in section 3.2.1 needs a little more context or definition or example. "
# paragraph = all_reviews_df.sample(1).summary_of_weaknesses.values[0]
# print(f"Paragraph: {paragraph}\n\n")
# points = split_into_points(paragraph)
# for i, point in enumerate(points, 1):
#     print(f"Point {i}: {point}")

In [16]:

df = pd.read_csv('../../data/reviewer2_ARR_2022_reviews_gemma2.csv')

df = filter_reviews.filter_reviews(df,'focused_review')

new_col = []
for i,r in tqdm(df.iterrows(),total=len(df)):
    focused_review = re.sub(r'\s+', ' ', r['focused_review'])
    splitted_reviews = split_into_points(focused_review)
    splitted_reviews = merge_short_sentences(splitted_reviews)

    
    # splitted_reviews = semantic_segmentation.split_paragraph(r['focused_review'])
    # print(r['focused_review'].strip())
    # print(''.join(splitted_reviews).strip())
    # if len(''.join(splitted_reviews).strip()) != len(r['focused_review'].strip()):
    #     print(len(''.join(splitted_reviews).strip()), len(r['focused_review'].strip()))
    #     print(r['focused_review'].strip())
    #     print(''.join(splitted_reviews).strip())

    if abs(len(''.join(splitted_reviews).strip().replace(' ','').replace('\n','')) - len(focused_review.strip().replace(' ','').replace('\n',''))) > 10:
        
        print(''.join(splitted_reviews).strip())
        print('\n\n')
        print(focused_review.strip())
        break
    # assert abs(len(''.join(splitted_reviews).strip()) - len(r['focused_review'].strip())) < 10
    new_col.append('$$$'.join(splitted_reviews))

df['split_review'] = new_col





filtering reviwes, and only considering the ones with length of one STD away form mean.
Number of the reviews before filtering: 573
mean: 211.1405109489051 std: 124.57447410982289 min: 86.56603683908222 max: 335.71498505872796
Number of the reviews after filtering: 374


100%|██████████| 374/374 [00:00<00:00, 2486.66it/s]


In [17]:
df.to_csv("../../data/reviewer2_ARR_2022_manual_split_reviews.csv")

In [18]:
df['split_review'].iloc[0]

'- The paper is not so clear as the introduction part does not show much background about the cognitive models. The authors may need to refine the writing.$$$- This paper shows lots of analyses about the attention weights of the Transformer models. Although it presents an in-depth analysis, how to utilize these conclusions is still unclear. If the authors can present a more powerful attention mechanism which correlates better with human attention and improves task performance compared with those baselines, I will be more convinced of the conclusions. Comments:$$$1. If a model correlates better with human attention compared to other models, does it show better task performance?$$$2. Does all the experiments keep the numbers of the parameters of the compared models the same? Typos: Line 255 / Line 259: p < .05 --> p < 0.05 Line 266: Correlations grouped by sentence length shows stable values around .6 (SST) and .4-.6 (Wikipedia) except for shorter sen- tences where correlations fluctuate

In [19]:
# with open('split_test_0.6.txt','w')as f:
#     for i,r in df.sample(5).iterrows():
#     # for i, r in df.iloc[:5].iterrows():
#         f.write(f'Focused review:\n\n{r["focused_review"]}\n\n')
#         for sr in r['split_review'].split('$$$'):
#             f.write(f'Review Point: {sr}\n')
#         f.write('='*50 + '\n\n')

        # f.write('Original Review:')
        # f.write('\n')
        # f.write(r['focused_review'])
        # f.write('\n')
        # f.write('$' * 50)
        # f.write('\n')
        # f.write('$' * 50)
        # f.write('\n')
        # f.write('Splitted Review:')
        # f.write('\n')
        # for l in r['split_review']:
        #     f.write(l)
        #     f.write('\n')
        #     f.write('-' * 50)
        #     f.write('\n')
        # f.write('#' * 70)
        # f.write('\n')
        

In [20]:
split_reviews = pd.read_csv("../../data/reviewer2_ARR_2022_split_reviews.csv")
# split_reviews = split_reviews.astype('object')

In [21]:
for i, row in split_reviews.iterrows():
    cur_split_review = row['split_review']
    cur_split_review = cur_split_review.split('$$$')    
    num_of_points = len(cur_split_review)
    print(num_of_points)
    break

10


In [22]:
cur_split_review

['- The paper is not so clear as the introduction part does not show much background about the cognitive models. ',
 'The authors may need to refine the writing.  ',
 '- This paper shows lots of analyses about the attention weights of the Transformer models. ',
 'Although it presents an in-depth analysis, how to utilize these conclusions is still unclear. ',
 'If the authors can present a more powerful attention mechanism which correlates better with human attention and improves task performance compared with those baselines, I will be more convinced of the conclusions. \n',
 'Comments: \xa01. \xa0If a model correlates better with human attention compared to other models, does it show better task performance?\n\xa0',
 '2. \xa0Does all the experiments keep the numbers of the parameters of the compared models the same?\n',
 'Typos: Line 255 / Line 259:  ',
 'p < .05 -->  p < 0.05 Line 266:  Correlations grouped by sentence length shows stable values around .6  (SST) and .4-.6 (Wikipedia)